In [3]:
from openai import OpenAI
import base64
import os
from typing import Literal

def encode_image(image_path: str) -> str:
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
    except FileNotFoundError:
        raise FileNotFoundError(f"Image file not found at path: {image_path}")
    except Exception as e:
        raise Exception(f"Error encoding image: {str(e)}")

def analyze_image_signal(
    image_path: str,
    api_key: str
) -> Literal['up', 'down', 'left', 'right', 'stop', 'no signal']:
    client = OpenAI(api_key=api_key)
    
    # Encode image
    base64_image = encode_image(image_path)
    
    # Define the prompt
    prompt = """Analyze this image and determine the direction of the signal shown in it. 
    The possible answers are: 'up', 'down', 'left', 'right', 'stop', or 'no signal'. 
    If you are unsure or the image does not contain a signal, return only 'no signal'. 
    Provide only the exact answer as a single word: 'up', 'down', 'left', 'right', 'stop', or 'no signal'. 
    Do not include any explanations or additional text."""

    try:
        # Make API call
        response = client.chat.completions.create(
    model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=10  # Limit response length since we only need a single word
        )
        
        # Extract and validate response
        result = response.choices[0].message.content.strip().lower()
        valid_responses = {'up', 'down', 'left', 'right', 'stop', 'no signal'}
        
        if result not in valid_responses:
            return 'no signal'
            
        return result
        
    except Exception as e:
        print(f"Error analyzing image: {str(e)}")
        return 'no signal'

# Example usage
if __name__ == "__main__":
    # Replace with your actual API key and image path
    api_key = "your-api-key"
    image_path = "Direction/Up/0.jpg"
    
    try:
        result = analyze_image_signal(image_path, api_key)
        print(f"Detected signal: {result}")
    except Exception as e:
        print(f"Error: {str(e)}")

Detected signal: up
